# Setup Workspace

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
cd drive/My Drive/Colab Notebooks

/content/drive/My Drive/Colab Notebooks


In [0]:
 !pip install -q kaggle

In [0]:
 !mkdir -p ~/.kaggle
 !cp kaggle.json ~/.kaggle/

In [0]:
!chmod 600 ~/.kaggle/kaggle.json

In [6]:
cd /content/

/content


In [7]:
!kaggle datasets download -d hakymulla/cgiar-computer-vision-for-crop-disease

100% 2.98G/3.00G [00:34<00:00, 75.0MB/s]
100% 3.00G/3.00G [00:34<00:00, 93.2MB/s]


In [0]:
%%capture
!unzip cgiar-computer-vision-for-crop-disease.zip
!rm cgiar-computer-vision-for-crop-disease.zip

In [0]:
%%capture
!pip install -U catalyst

In [0]:
# a gif image is causing some problem while training so let's get rid of that
!rm ICLR/train/train/leaf_rust/*.gif

# Data preprocessing

In [17]:
import catalyst
from pathlib import Path
from catalyst.dl import utils

SEED = 42
utils.set_global_seed(SEED)

/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning:

numpy.ufunc size changed, may indicate binary incompatibility. Expected 216, got 192

/usr/lib/python3.6/importlib/_bootstrap.py:219: ImportWarning:

can't resolve package from __spec__ or __package__, falling back on __name__ and __path__

/usr/local/lib/python3.6/dist-packages/tensorflow_core/python/keras/backend.py:5747: ResourceWarning:

unclosed file <_io.TextIOWrapper name='/root/.keras/keras.json' mode='r' encoding='UTF-8'>



In [0]:
ROOT = "ICLR/"

In [14]:
from catalyst.utils import (
    create_dataset, create_dataframe, get_dataset_labeling, map_dataframe
)
dataset = create_dataset(dirs=f"{ROOT}train/train/*", extension="*")
df = create_dataframe(dataset, columns=["class", "filepath"])

tag_to_label = get_dataset_labeling(df, "class")
class_names = [
    name for name, id_ in sorted(tag_to_label.items(), key=lambda x: x[1])
]

df_with_labels = map_dataframe(
    df, 
    tag_column="class", 
    class_column="label", 
    tag2class=tag_to_label, 
    verbose=False
)
df_with_labels.head()

,class,filepath,label
0,healthy_wheat,ICLR/train/train/healthy_wheat/03TD19.jfif,0
1,healthy_wheat,ICLR/train/train/healthy_wheat/0LBIWV.jpg,0
2,healthy_wheat,ICLR/train/train/healthy_wheat/0O5BON.jfif,0
3,healthy_wheat,ICLR/train/train/healthy_wheat/0PFX47.jpg,0
4,healthy_wheat,ICLR/train/train/healthy_wheat/1LNMUQ.jfif,0


In [15]:
df_with_labels.shape

(875, 3)

In [18]:
df_with_labels = catalyst.utils.pandas.balance_classes(df_with_labels, class_column='label',
                                                    random_state=SEED, how='upsampling')
print(df_with_labels.shape)
df_with_labels.head()

(1128, 3)


,class,filepath,label
0,healthy_wheat,ICLR/train/train/healthy_wheat/03TD19.jfif,0
1,healthy_wheat,ICLR/train/train/healthy_wheat/0LBIWV.jpg,0
2,healthy_wheat,ICLR/train/train/healthy_wheat/0O5BON.jfif,0
3,healthy_wheat,ICLR/train/train/healthy_wheat/0PFX47.jpg,0
4,healthy_wheat,ICLR/train/train/healthy_wheat/1LNMUQ.jfif,0


In [19]:
df_with_labels.loc[:,'kfold'] = -1
df_with_labels.head()

,class,filepath,label,kfold
0,healthy_wheat,ICLR/train/train/healthy_wheat/03TD19.jfif,0,-1
1,healthy_wheat,ICLR/train/train/healthy_wheat/0LBIWV.jpg,0,-1
2,healthy_wheat,ICLR/train/train/healthy_wheat/0O5BON.jfif,0,-1
3,healthy_wheat,ICLR/train/train/healthy_wheat/0PFX47.jpg,0,-1
4,healthy_wheat,ICLR/train/train/healthy_wheat/1LNMUQ.jfif,0,-1


In [20]:
df_with_labels = df_with_labels.sample(frac=1).reset_index(drop=True)
df_with_labels.head()

,class,filepath,label,kfold
0,stem_rust,ICLR/train/train/stem_rust/1CQ95E.JPG,2,-1
1,stem_rust,ICLR/train/train/stem_rust/7QBIPY.jpg,2,-1
2,leaf_rust,ICLR/train/train/leaf_rust/1Z2P44.jpg,1,-1
3,healthy_wheat,ICLR/train/train/healthy_wheat/ZB9CAK.jpg,0,-1
4,stem_rust,ICLR/train/train/stem_rust/4PS17R.jpg,2,-1


In [0]:
from sklearn.model_selection import StratifiedKFold
import numpy as np

X = df_with_labels.filepath.values
y = df_with_labels.label.values

skf = StratifiedKFold(n_splits=5)

for fold, (train_index, val_index) in enumerate(skf.split(X, y)):
    #print("TRAIN:", train_index, "TEST:", val_index)
    df_with_labels.loc[val_index, 'kfold'] = fold

In [22]:
df_with_labels.kfold.value_counts()

2    226
1    226
0    226
4    225
3    225
Name: kfold, dtype: int64

In [23]:
df_with_labels.head()

,class,filepath,label,kfold
0,stem_rust,ICLR/train/train/stem_rust/1CQ95E.JPG,2,0
1,stem_rust,ICLR/train/train/stem_rust/7QBIPY.jpg,2,0
2,leaf_rust,ICLR/train/train/leaf_rust/1Z2P44.jpg,1,0
3,healthy_wheat,ICLR/train/train/healthy_wheat/ZB9CAK.jpg,0,0
4,stem_rust,ICLR/train/train/stem_rust/4PS17R.jpg,2,0


In [24]:
df_with_labels.shape

(1128, 4)

In [0]:
df_with_labels.to_csv('labels.csv', index=False)